In [12]:
# pip install selenium

In [13]:
#pip install webdriver-manager

In [1]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def save_json(path, data):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)


def get_people_pages(agent):
    links = agent.find_elements(By.XPATH, '//tbody/tr/td/a')

    return [link.get_attribute('href') for link in links]


def get_person_info(agent, url):
    agent.get(url)
    skils= agent.find_elements(By.XPATH, '//span[@class="skills-item"]')

    return {
        'name': agent.find_element(By.XPATH, '//div[@class="person-name"]/p').text,
        'gender': agent.find_element(By.XPATH, '//span[@class="gender"]').text,
        'age': agent.find_element(By.XPATH, '//span[@class="age"]').text,  # !!!
        'skills': [skil.text for skil in skils],  # !!!
        'previous_company': agent.find_element(By.XPATH, '//span[@class="previous-company"]').text,
        'desired_salary': agent.find_element(By.XPATH, '//span[@class="salary"]').text,
        'location': {
            'lat': agent.find_elements(By.XPATH, '//span[@class="lat"]')[0].text,
            'lon': agent.find_elements(By.XPATH, '//span[@class="lat"]')[1].text
        },
        'register_date': agent.find_element(By.XPATH, '//span[@class="registered"]').text  # !!!
    }


if __name__ == '__main__':
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    driver.get('https://iirmephi.ru/tutorials/jobhunt/')

    pages = []
    while True:
        try:
            pages += get_people_pages(driver)
            next_button = driver.find_element(By.XPATH, '//li[@class="next"]/a')
            next_button.click()
        except NoSuchElementException as err:
            break

    result = []
    for page in pages:
        info = get_person_info(driver, page)
        result.append(info)
    save_json(f'result.json', result)

    driver.close()


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:00<00:00, 9.97MB/s]
